colab notebook link - https://colab.research.google.com/drive/1yPhA40udmtcs9uNNnBk3L15_exRDBtkc?usp=sharing

In [27]:
import pandas as pd
import altair as alt

In [28]:
# Load the datasets
try:
    df_trades = pd.read_csv("historical_data.csv")
    df_sentiment = pd.read_csv("fear_greed_index.csv")

    # Inspect Trader Data
    print("--- Historical Trader Data Info ---")
    df_trades.info()
    print("\n--- Historical Trader Data Head ---")
    print(df_trades.head())

    # Inspect Sentiment Data
    print("\n--- Bitcoin Market Sentiment Data Info ---")
    df_sentiment.info()
    print("\n--- Bitcoin Market Sentiment Data Head ---")
    print(df_sentiment.head())

except FileNotFoundError:
    print("Error: Make sure you have uploaded both 'historical_data.csv' and 'fear_greed_index.csv'")
except Exception as e:
    print(f"An error occurred: {e}")

--- Historical Trader Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211224 entries, 0 to 211223
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Account           211224 non-null  object 
 1   Coin              211224 non-null  object 
 2   Execution Price   211224 non-null  float64
 3   Size Tokens       211224 non-null  float64
 4   Size USD          211224 non-null  float64
 5   Side              211224 non-null  object 
 6   Timestamp IST     211224 non-null  object 
 7   Start Position    211224 non-null  float64
 8   Direction         211224 non-null  object 
 9   Closed PnL        211224 non-null  float64
 10  Transaction Hash  211224 non-null  object 
 11  Order ID          211224 non-null  int64  
 12  Crossed           211224 non-null  bool   
 13  Fee               211224 non-null  float64
 14  Trade ID          211224 non-null  float64
 15  Timestamp         211224 non-nul

#Data Cleaning

In [29]:
# ---Prepare Trader Data (df_trades) ---
# Converting 'Timestamp IST' (string) to a proper datetime object
# format='%d-%m-%Y %H:%M' tells pandas how to read the date string

df_trades['datetime'] = pd.to_datetime(df_trades['Timestamp IST'], format='%d-%m-%Y %H:%M')

# Extracting just the date (ignoring time) to use as a key for merging

df_trades['date_key'] = df_trades['datetime'].dt.date

In [30]:
df_trades.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp,datetime,date_key
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,2024-12-02 22:50:00,2024-12-02
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12,2024-12-02 22:50:00,2024-12-02
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12,2024-12-02 22:50:00,2024-12-02
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12,2024-12-02 22:50:00,2024-12-02
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12,2024-12-02 22:50:00,2024-12-02


In [31]:
# --- Prepare Sentiment Data (df_sentiment) ---
# Converting 'date' (string) to a proper datetime object
df_sentiment['datetime'] = pd.to_datetime(df_sentiment['date'], format='%Y-%m-%d')

# Extracting just the date to use as a key
df_sentiment['date_key'] = df_sentiment['datetime'].dt.date

In [32]:
df_sentiment.head()

,timestamp,value,classification,date,datetime,date_key
0,1517463000,30,Fear,2018-02-01,2018-02-01,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02,2018-02-02,2018-02-02
2,1517635800,40,Fear,2018-02-03,2018-02-03,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04,2018-02-04,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05,2018-02-05,2018-02-05


In [33]:
# --- Merging the datasets ---
# We perform a 'left' merge, keeping every trade from df_trades
# and attaching the matching sentiment data ('value', 'classification') for its date.
df_merged = pd.merge(
    df_trades,
    df_sentiment[['date_key', 'value', 'classification']],
    on='date_key',
    how='left'
)

In [34]:
df_merged.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp,datetime,date_key,value,classification
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,2024-12-02 22:50:00,2024-12-02,80.0,Extreme Greed
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12,2024-12-02 22:50:00,2024-12-02,80.0,Extreme Greed
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12,2024-12-02 22:50:00,2024-12-02,80.0,Extreme Greed
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12,2024-12-02 22:50:00,2024-12-02,80.0,Extreme Greed
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12,2024-12-02 22:50:00,2024-12-02,80.0,Extreme Greed


In [35]:
# --- Cleaning the Merged Data ---
# Dropping any trades that didn't have a matching sentiment score

df_merged.dropna(subset=['value', 'classification'], inplace=True)

print("\n--- Merged Data Head ---")
print(df_merged[['datetime', 'date_key', 'Side', 'Size USD', 'value', 'classification']].head())


--- Merged Data Head ---
             datetime    date_key Side  Size USD  value classification
0 2024-12-02 22:50:00  2024-12-02  BUY   7872.16   80.0  Extreme Greed
1 2024-12-02 22:50:00  2024-12-02  BUY    127.68   80.0  Extreme Greed
2 2024-12-02 22:50:00  2024-12-02  BUY   1150.63   80.0  Extreme Greed
3 2024-12-02 22:50:00  2024-12-02  BUY   1142.04   80.0  Extreme Greed
4 2024-12-02 22:50:00  2024-12-02  BUY     69.75   80.0  Extreme Greed


#Feature Engineering & Aggregation

In [36]:
# Feature Engineering: Net Flow ---
# Create a new column 'net_flow_usd'
# If the 'Side' was 'BUY', the value is positive 'Size USD'
# If the 'Side' was 'SELL', the value is negative 'Size USD'

df_merged['net_flow_usd'] = df_merged.apply(
    lambda row: row['Size USD'] if row['Side'] == 'BUY' else -row['Size USD'],
    axis=1
)

In [37]:
df_merged.sample(5)

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,...,Order ID,Crossed,Fee,Trade ID,Timestamp,datetime,date_key,value,classification,net_flow_usd
42151,0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,BTC,42456.000,0.01178,500.13,SELL,06-02-2024 02:50,-1.441230,Open Short,0.000000,...,8900882080,True,0.125032,8.970000e+14,1.710000e+12,2024-02-06 02:50:00,2024-02-06,64.0,Greed,-500.13
122281,0x47add9a56df66b524d5e2c1993a43cde53b6ed85,PENGU,0.028,3756.00000,105.17,SELL,18-12-2024 04:19,-26955.000000,Open Short,0.000000,...,56114462834,False,0.010095,1.020000e+15,1.730000e+12,2024-12-18 04:19:00,2024-12-18,81.0,Extreme Greed,-105.17
140496,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,ETH,2191.600,0.50000,1095.80,BUY,08-03-2025 00:35,3.000000,Open Long,0.000000,...,78029427942,True,0.383530,2.350000e+14,1.740000e+12,2025-03-08 00:35:00,2025-03-08,28.0,Fear,1095.80
169128,0xbee1707d6b44d4d52bfe19e41f8a828645437aab,@107,25.621,29.27000,749.93,BUY,16-02-2025 09:01,337.116492,Buy,0.000000,...,72615138500,True,0.010244,4.390000e+14,1.740000e+12,2025-02-16 09:01:00,2025-02-16,54.0,Neutral,749.93
27205,0x4f93fead39b70a1824f981a54d4e55b278e9f760,BTC,97559.000,0.00015,14.63,BUY,16-02-2025 01:12,-0.614640,Close Short,0.008745,...,72564312151,True,0.004390,1.960000e+14,1.740000e+12,2025-02-16 01:12:00,2025-02-16,54.0,Neutral,14.63


In [38]:
# --- Aggregate Trade Data by Day ---
# We group by the date and its corresponding sentiment data

df_daily_analysis = df_merged.groupby(['date_key', 'value', 'classification']).agg(
    total_volume_usd=('Size USD', 'sum'),      # Total $ value traded that day
    net_pnl=('Closed PnL', 'sum'),           # Total profit/loss of all traders
    trade_count=('Trade ID', 'count'),       # Total number of trades
    net_flow_usd=('net_flow_usd', 'sum')     # Net buy/sell imbalance for the day
).reset_index()

In [39]:
df_daily_analysis.head()

,date_key,value,classification,total_volume_usd,net_pnl,trade_count,net_flow_usd
0,2023-05-01,63.0,Greed,477.00,0.000000,3,477.00
1,2023-12-05,75.0,Extreme Greed,50005.83,0.000000,9,8.59
2,2023-12-14,72.0,Greed,113203.35,-205.434737,11,-10388.41
3,2023-12-15,70.0,Greed,10609.95,-24.632034,2,10609.95
4,2023-12-16,67.0,Greed,15348.77,0.000000,3,15348.77


In [40]:
# Converting date_key from 'datetime.date' object to a string
# This makes the DataFrame JSON serializable for Altair.

df_daily_analysis['date_key'] = df_daily_analysis['date_key'].astype(str)

In [41]:
# Prepare for Plotting
# Defining the logical order for our sentiment categories

sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']

In [42]:
# Creating a new column that's an 'ordered categorical' type.
# This tells Altair to plot them in this order, not alphabetically.

df_daily_analysis['classification_ordered'] = pd.Categorical(
    df_daily_analysis['classification'],
    categories=sentiment_order,
    ordered=True
)

In [43]:
# Creating a proper datetime column (from the string 'date_key') for time-series plotting

df_daily_analysis['date'] = pd.to_datetime(df_daily_analysis['date_key'])

In [44]:
print("\n--- Daily Aggregated Analysis Data Head (Corrected) ---")
print(df_daily_analysis.head())

print("\n--- Daily Aggregated Analysis Info (Corrected) ---")
df_daily_analysis.info()


--- Daily Aggregated Analysis Data Head (Corrected) ---
     date_key  value classification  total_volume_usd     net_pnl  \
0  2023-05-01   63.0          Greed            477.00    0.000000   
1  2023-12-05   75.0  Extreme Greed          50005.83    0.000000   
2  2023-12-14   72.0          Greed         113203.35 -205.434737   
3  2023-12-15   70.0          Greed          10609.95  -24.632034   
4  2023-12-16   67.0          Greed          15348.77    0.000000   

   trade_count  net_flow_usd classification_ordered       date  
0            3        477.00                  Greed 2023-05-01  
1            9          8.59          Extreme Greed 2023-12-05  
2           11     -10388.41                  Greed 2023-12-14  
3            2      10609.95                  Greed 2023-12-15  
4            3      15348.77                  Greed 2023-12-16  

--- Daily Aggregated Analysis Info (Corrected) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (

In [45]:
df_daily_analysis.to_csv('daily_trade_analysis.csv', index=False)

#EDA

##Volume Vs Sentiment

In [46]:
# Creating a boxplot to show the distribution of daily volume for each sentiment category
chart_volume = alt.Chart(df_daily_analysis).mark_boxplot(extent='min-max').encode(
    # X-axis: Sentiment Classification, sorted using our defined order
    x=alt.X('classification_ordered', sort=sentiment_order, title='Sentiment'),

    # Y-axis: Total Daily Volume in USD
    y=alt.Y('total_volume_usd', title='Total Daily Volume (USD)'),


    color=alt.Color('classification_ordered', title="Sentiment", legend=None),

    tooltip=['classification_ordered', 'total_volume_usd']
).properties(
    title='Distribution of Daily Trading Volume by Market Sentiment'
).interactive()

chart_volume

alt.Chart(...)

### Analysis of Volume-Sentiment Divergence

1.  **"Extreme Fear" (High-Volume Anomaly):**
    * **Observation:** This phase correlates with the highest median trading volume.
    * **Insight:** This is indicative of a **capitulation event**. The high volume is driven by mass panic-selling and involuntary liquidations, representing the point of maximum financial pain and holder turnover.

2.  **"Greed" (Low-Volume Anomaly):**
    * **Observation:** This phase correlates with the *lowest* median trading volume.
    * **Insight:** This signals **market complacency**. The price rally is not supported by a high-conviction inflow of new capital or broad participation. Such low-volume uptrends are structurally weak and highly vulnerable to reversal.

3.  **"Extreme Greed" (Climax Volume):**
    * **Observation:** Volume is higher than in the "Greed" phase but significantly lower than during "Extreme Fear."
    * **Insight:** This likely represents the **"buying climax" or "blow-off top."** It's the final surge of FOMO-driven retail participation, which is insufficient to match the volume signature of a market-wide panic.



### **Corrected Key Takeaway & Strategy**

* **Key Divergence:** A critical divergence exists between sentiment and volume. Peak market activity is driven by panic-induced capitulation (Fear), not by FOMO-driven accumulation (Greed).
* **Strategic Implication:** A contrarian strategy is supported. The point of maximum activity (Extreme Fear) signals a potential accumulation zone. Conversely, the point of minimum activity (Greed) signals a structurally weak rally and a high-risk environment for new long positions.

##Profitability (PnL) vs. Sentiment

In [47]:
# We must first aggregate the daily PnL into total PnL for each category
pnl_by_sentiment = df_daily_analysis.groupby('classification_ordered')['net_pnl'].sum().reset_index(name='total_net_pnl')

chart_pnl = alt.Chart(pnl_by_sentiment).mark_bar().encode(
    x=alt.X('classification_ordered', sort=sentiment_order, title='Sentiment'),
    y=alt.Y('total_net_pnl', title='Total Net Profit & Loss (USD)'),
    color=alt.Color('classification_ordered', title="Sentiment", legend=None),
    tooltip=['classification_ordered', 'total_net_pnl']
).properties(
    title='Total Net PnL by Market Sentiment'
).interactive()

chart_pnl

/tmp/ipython-input-3096549044.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pnl_by_sentiment = df_daily_analysis.groupby('classification_ordered')['net_pnl'].sum().reset_index(name='total_net_pnl')


alt.Chart(...)

### Analysis of PnL vs. Sentiment Graph

1.  **Optimal Profitability in "Fear":** The highest net PnL is generated during "Fear" phases. This suggests the optimal strategy is not "buying the panic" but trading the sustained volatility or clearer trends (e.g., short-selling, buying dips) that emerge during a general fearful market state.

2.  **High-Profitability in "Extreme Greed":** The second-highest PnL in "Extreme Greed" indicates that momentum-based, trend-following strategies are also highly effective, successfully capturing upside during euphoric market phases.

3.  **"Extreme Fear" is Least Profitable:** This is a major divergence from classic contrarian theory. The data shows that "Extreme Fear" is a *low-profitability trap*, where the extreme volatility, illiquidity, or psychological pressure prevents effective capital deployment.

4.  **Low-Profitability in "Neutral" / "Greed":** The muddled "middle" of the sentiment spectrum ("Greed," "Neutral") offers the lowest profitability. This implies that market ambiguity, complacency, or lack of a clear emotional driver results in a poor risk/reward environment.

The data shows two distinct, profitable strategies: **1) contrarian trading within the "Fear" zone** and **2) momentum trading within the "Extreme Greed" zone.** It explicitly refutes the "buy the blood" maxim, identifying "Extreme Fear" as the *least* profitable regime. Profitability is maximized in high-conviction, high-emotion (but not peak-panic) environments and minimized in ambiguous, low-emotion states.

##Trading Direction (Net Flow) vs. Sentiment

In [48]:
# Calculate the average daily net flow for each sentiment category
avg_flow_by_sentiment = df_daily_analysis.groupby('classification_ordered')['net_flow_usd'].mean().reset_index(name='average_daily_net_flow')

chart_flow = alt.Chart(avg_flow_by_sentiment).mark_bar().encode(
    x=alt.X('classification_ordered', sort=sentiment_order, title='Sentiment'),
    y=alt.Y('average_daily_net_flow', title='Average Daily Net Flow (USD)'),
    color=alt.Color('classification_ordered', title="Sentiment", legend=None),
    tooltip=['classification_ordered', 'average_daily_net_flow']
).properties(
    title='Average Daily Net Flow (Buy/Sell Imbalance) by Market Sentiment'
).interactive()

chart_flow

/tmp/ipython-input-2382580293.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_flow_by_sentiment = df_daily_analysis.groupby('classification_ordered')['net_flow_usd'].mean().reset_index(name='average_daily_net_flow')


alt.Chart(...)

###Analysis of Net Flow vs. Sentiment

1.  **"Fear" (Peak Net-Inflow):** This phase shows maximum accumulation. This is a strong contrarian signal: capital is deployed aggressively into market weakness, pre-empting the "Extreme Fear" bottom.

2.  **"Greed" (Secondary Net-Inflow):** This phase shows continued accumulation, indicative of a trend-following strategy that adds to positions as positive sentiment builds.

3.  **"Neutral" (Peak Net-Outflow):** This is the most significant anomaly. The largest net-outflow occurs during market indecision. This suggests a proactive profit-taking or derisking strategy, where liquidity is aggressively reduced *before* an extreme is reached.

4.  **"Extreme Greed" (Net-Outflow):** This is a classic contrarian distribution phase. Traders are net-sellers, providing liquidity and realizing profits during peak market euphoria.

5.  **"Extreme Fear" (Net-Outflow):** This phase shows a net-outflow, aligning with herd capitulation. This likely represents a risk-management-driven action, such as stop-loss execution or cutting losing positions.

### Hybrid Strategy

The data indicates a sophisticated hybrid strategy, not a simple contrarian or momentum model.
* **Accumulation** is two-pronged: contrarian (peak buying in "Fear") and trend-following (secondary buying in "Greed").
* **Distribution** is also two-pronged: contrarian (selling into "Extreme Greed") and anomalously, a primary exit during "Neutral" phases.
* **Risk Management** is evident, with capitulation-aligned selling during "Extreme Fear."

Time Series Divergence

In [49]:
# Base chart for date selection
base = alt.Chart(df_daily_analysis).encode(
    # Use the 'date' column (a proper datetime object) for the X-axis
    x=alt.X('date', title='Date', axis=alt.Axis(format="%Y-%m-%d"))
).properties(
    width=800,
    height=300
)

# Line for Sentiment (Bitcoin Fear & Greed Index)
line_sentiment = base.mark_line(color='blue', opacity=0.7).encode(
    y=alt.Y('value', title='Sentiment Score (Fear/Greed)', scale=alt.Scale(zero=False)),
    tooltip=['date', alt.Tooltip('value', title='Sentiment Score')]
).properties(
    title='Market Sentiment vs. Net Trading Flow Over Time'
)

# Line for Net Flow (Green)
line_flow = base.mark_line(color='green', opacity=0.7).encode(
    y=alt.Y('net_flow_usd', title='Net Trading Flow (USD)', scale=alt.Scale(zero=False)),
    tooltip=['date', alt.Tooltip('net_flow_usd', title='Net Flow USD')]
)

# Combine the two lines on a dual-axis chart
chart_timeseries = alt.layer(line_sentiment, line_flow).resolve_scale(
    y='independent'
).interactive()
chart_timeseries

alt.LayerChart(...)

### Analysis of Sentiment vs. Net Flow Time-Series

This graph plots market sentiment against capital net flow, revealing critical divergences.

1.  **Bearish Divergence (Top Formation):**
    * **Observation:** In Feb-Mar 2025, the sentiment indicator registered a "double top" or sustained euphoria. Concurrently, the net flow (capital action) failed to confirm the second peak, showing significantly diminished net inflows.
    * **Insight:** This divergence signals a decoupling of market psychology from capital action. It indicates that conviction is failing and distribution is occurring under the cover of market euphoria, a classic precursor to a price correction.

2.  **Bullish Divergence (Bottom Formation):**
    * **Observation:** In mid-March 2025, as sentiment capitulated to its nadir ("Extreme Fear"), net flow (which was negative) failed to post a new low, instead forming a higher low.
    * **Insight:** This non-confirmation signals seller exhaustion. At the point of maximum pessimism, the actual selling pressure had already abated, indicating that the capitulation event was largely complete and a market bottom was forming.

3.  **Congruent Periods (Alignment):**
    * **Observation:** Other periods (e.g., Jan 2025) show a high correlation where sentiment and net flow move in tandem (e.g., fear/selling, greed/buying).
    * **Insight:** This represents typical "herd" or aligned market behavior, which is non-divergent and provides no edge.

The graph provides clear evidence that market-turning points are signaled by **divergences** between sentiment (psychology) and net flow (action).
* **A distribution signal** is generated when sentiment *fails* to be confirmed by net flow during "Greed."
* **An accumulation signal** is generated when selling flow *fails* to confirm peak pessimism during "Fear."